In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1, bias = False) # param =  #input -? OUtput? RF
        self.batchN1 = nn.BatchNorm2d(32)


        self.conv2 = nn.Conv2d(32, 32, 3, padding=1, bias = False) # param =
        self.batchN2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)


        self.conv3 = nn.Conv2d(32, 16, 3, padding=1, bias = False) # param =
        self.batchN3 = nn.BatchNorm2d(16)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(16, 16, 3, bias = False) #

        self.conv5 = nn.Conv2d(16, 16, 3, bias = False)

        self.gap1 = nn.AdaptiveAvgPool2d(1)

        self.fc1 = nn.Linear(16, 10)

        self.drop = nn.Dropout2d(0.05)


    def forward(self, x):
        #L 1
        #print("x shape1", x.shape)
        x = self.batchN1(F.relu(self.conv1(x)))
        x = self.drop(x)

        # L 2
        #print("x shape2", x.shape)
        x = self.pool2(self.batchN2(F.relu(self.conv2(x))))
        x = self.drop(x)

        # L 3
        #print("x shape3", x.shape)
        x = F.relu(self.conv3(x))
        #print("x shape4", x.shape)
        self.batchN3(x)
        #print("x shape5", x.shape)
        x = self.pool3(x)

        # L 4
        #print("x shape6", x.shape)
        x = F.relu(self.conv4(x))

        # L 5
        #print("x shape6", x.shape)
        x = F.relu(self.conv5(x))

        # L 6
        #print("x shape1", x.shape)
        x = self.gap1(x)

        #print("x shape2", x.shape)
        x = x.view(-1, 16)

        #print("x shape9", x.shape)
        x = self.fc1(x)

        #print("\nx shape4", x.shape)
        return F.log_softmax(x, dim = 1)

In [48]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
       BatchNorm2d-2           [-1, 32, 28, 28]              64
         Dropout2d-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 32, 28, 28]           9,216
       BatchNorm2d-5           [-1, 32, 28, 28]              64
         MaxPool2d-6           [-1, 32, 14, 14]               0
         Dropout2d-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           4,608
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        MaxPool2d-10             [-1, 16, 7, 7]               0
           Conv2d-11             [-1, 16, 5, 5]           2,304
           Conv2d-12             [-1, 16, 3, 3]           2,304
AdaptiveAvgPool2d-13             [-1, 16, 1, 1]               0
           Linear-14                   

In [49]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [50]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch: {epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [51]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

num_epochs = 20

for epoch in range(1, num_epochs):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch: 1 loss=0.11182896047830582 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.78it/s]



Test set: Average loss: 0.0809, Accuracy: 9743/10000 (97.43%)



epoch: 2 loss=0.012070861645042896 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.66it/s]



Test set: Average loss: 0.0453, Accuracy: 9867/10000 (98.67%)



epoch: 3 loss=0.06207633018493652 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.05it/s]


...


Test set: Average loss: 0.0272, Accuracy: 9925/10000 (99.25%)



epoch: 18 loss=0.004796942695975304 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.20it/s]



Test set: Average loss: 0.0210, Accuracy: 9936/10000 (99.36%)



epoch: 19 loss=0.0016453162534162402 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 26.39it/s]



Test set: Average loss: 0.0206, Accuracy: 9935/10000 (99.35%)

